In [3]:
!pip install openai streamlit pytesseract pillow reportlab speechrecognition



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install streamlit google-generativeai python-dotenv pytesseract pillow reportlab SpeechRecognition


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install --upgrade google-generativeai


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
#####################
# checking the models
#####################

import google.generativeai as genai
import os
from dotenv import load_dotenv

load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

print("✅ Available Models:")
for model in genai.list_models():
    print(f"🔹 {model.name} → {model.supported_generation_methods}")


✅ Available Models:
🔹 models/embedding-gecko-001 → ['embedText', 'countTextTokens']
🔹 models/gemini-1.5-pro-latest → ['generateContent', 'countTokens']
🔹 models/gemini-1.5-pro-002 → ['generateContent', 'countTokens', 'createCachedContent']
🔹 models/gemini-1.5-pro → ['generateContent', 'countTokens']
🔹 models/gemini-1.5-flash-latest → ['generateContent', 'countTokens']
🔹 models/gemini-1.5-flash → ['generateContent', 'countTokens']
🔹 models/gemini-1.5-flash-002 → ['generateContent', 'countTokens', 'createCachedContent']
🔹 models/gemini-1.5-flash-8b → ['createCachedContent', 'generateContent', 'countTokens']
🔹 models/gemini-1.5-flash-8b-001 → ['createCachedContent', 'generateContent', 'countTokens']
🔹 models/gemini-1.5-flash-8b-latest → ['createCachedContent', 'generateContent', 'countTokens']
🔹 models/gemini-2.5-pro-preview-03-25 → ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
🔹 models/gemini-2.5-flash-preview-05-20 → ['generateContent', 'countTokens',

In [7]:
#######################
#importing the data
#########################

import os
from dotenv import load_dotenv
import google.generativeai as genai
from reportlab.pdfgen import canvas


# Load .env file
load_dotenv()

# Get Gemini API Key
api_key = os.getenv("GEMINI_API_KEY")

# Configure Gemini API
genai.configure(api_key=api_key)


In [8]:
# ==============================
# STEP 2: Gemini Functions
# ==============================
def generate_questions(role):
    """Generate interview questions using Gemini"""
    prompt = f"""
    You are an interviewer. Generate 1 technical or behavioral interview questions
    for a candidate applying for {role}. Only return the list of questions as plain text, no explanation.
    """
    model = genai.GenerativeModel("gemini-2.5-pro")
    response = model.generate_content(prompt)
    return response.text

# def evaluate_answer(question, answer):
#     """Evaluate candidate answer using Gemini"""
#     eval_prompt = f"""
#     Evaluate this interview response:
#     Question: {question}
#     Answer: {answer}

#     Provide:
#     1. Score out of 10
#     2. Strengths
#     3. Weaknesses
#     4. Improvement Tips
#     """
#     model = genai.GenerativeModel("models/gemini-2.5-pro")
#     response = model.generate_content(eval_prompt)
#     return response.text

def evaluate_answer(question, answer):
    """Evaluate candidate answer using Gemini"""

    # Ensure inputs are not empty
    if not question.strip() or not answer.strip():
        return "⚠️ Question or Answer is empty. Please provide valid input."

    # Prompt to send to Gemini
    eval_prompt = f"""
    Evaluate this interview response:
    Question: {question}
    Answer: {answer}

    Provide:
    1. Score out of 10
    2. Strengths
    3. Weaknesses
    4. Improvement Tips
    """

    # Initialize model
    model = genai.GenerativeModel("gemini-2.5-pro")
    
    # Generate response
    response = model.generate_content(eval_prompt)

    # Safely access model output
    if response.candidates and response.candidates[0].content.parts:
        return response.candidates[0].content.parts[0].text
    else:
        return "⚠️ No feedback was generated by the model. Try rephrasing your answer."


In [9]:
# ==============================
# STEP 3: Input Modes
# ==============================
def get_text_input():
    return input("Type your answer: ")

def get_voice_input():
    """Capture voice and convert to text"""
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("🎤 Speak now...")
        audio = r.listen(source)
    try:
        return r.recognize_google(audio)
    except:
        return "Could not understand audio."

def get_image_input():
    """Extract text from image using OCR"""
    image_path = input("Enter image file path: ")
    img = Image.open(image_path)
    return pytesseract.image_to_string(img)


In [10]:
# ==============================
# STEP 4: PDF Report
# ==============================
# def create_report(filename, feedback):
#     c = canvas.Canvas(filename)
#     c.drawString(100, 750, "AI Interview Feedback Report")
#     y = 720
#     for line in feedback.split("\n"):
#         c.drawString(100, y, line)
#         y -= 20
#     c.save()
#     print(f"✅ Report saved as {filename}")


from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from textwrap import wrap

def create_report(filename, feedback):
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter
    
    # Margins
    left_margin = 50
    top_margin = height - 50
    
    # Title
    c.setFont("Helvetica-Bold", 16)
    c.drawString(left_margin, top_margin, "AI Interview Feedback Report")
    
    # Prepare text object
    text = c.beginText()
    text.setTextOrigin(left_margin, top_margin - 40)
    text.setFont("Helvetica", 12)
    
    # Wrap long lines
    max_width = 80  # number of characters per line
    wrapped_lines = []
    for line in feedback.split("\n"):
        wrapped_lines.extend(wrap(line, width=max_width))
        wrapped_lines.append("")  # blank line between sections

    # Add lines to PDF
    for line in wrapped_lines:
        text.textLine(line)

    c.drawText(text)
    c.save()


In [11]:
# ==============================
# STEP 5: Main Logic
# ==============================
def main():
    print("\n=== AI Multi-Modal Interview Bot (Gemini) ===")
    role = input("Enter Job Role: ")

    # Generate questions
    print("\nGenerating interview questions...")
    questions = generate_questions(role)
    print("\n--- Interview Questions ---")
    print(questions)

    # Choose input mode
    print("\nSelect Input Mode:")
    print("1. Text")
    print("2. Voice")
    print("3. Image")
    choice = input("Enter choice (1/2/3): ")

    if choice == "1":
        answer = get_text_input()
    elif choice == "2":
        answer = get_voice_input()
    elif choice == "3":
        answer = get_image_input()
    else:
        print("Invalid choice!")
        return

    # Evaluate answer
    print("\nEvaluating your answer...")
    feedback = evaluate_answer("Sample Question", answer)
    print("\n--- AI Feedback ---")
    print(feedback)

    # Save PDF report
    create_report("Interview_Report.pdf", feedback)
    print("\n✅ Interview completed! Check 'Interview_Report.pdf' for details.")

if __name__ == "__main__":
    main()


=== AI Multi-Modal Interview Bot (Gemini) ===

Generating interview questions...

--- Interview Questions ---
Describe the most technically challenging project you've worked on. What was your specific role, what were the major challenges, and how did you overcome them?

Select Input Mode:
1. Text
2. Voice
3. Image

Evaluating your answer...

--- AI Feedback ---
This is an excellent, well-structured interview response. It's a prime example of how to answer a technical behavioral question effectively.

### 1. Score: 9.5/10

This is a top-tier answer that would impress almost any interviewer. It's clear, concise, technically detailed, and demonstrates strong problem-solving skills and ownership.

---

### 2. Strengths

*   **Excellent Structure:** The response is perfectly structured, following a variant of the STAR method (Situation/Task, Action, Result) by explicitly breaking down Role, Challenges, Actions (How I Overcame Them), and Result. This makes it incredibly easy for the intervi

In [12]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
from reportlab.pdfgen import canvas


# Load .env file
load_dotenv()

# Get Gemini API Key
api_key = os.getenv("GEMINI_API_KEY")

# Configure Gemini API
genai.configure(api_key=api_key)
# ==============================
# STEP 2: Gemini Functions
# ==============================
def generate_questions(role):
    """Generate interview questions using Gemini"""
    prompt = f"""
    You are an interviewer. Generate 1 technical or behavioral interview questions
    for a candidate applying for {role}. Only return the list of questions as plain text, no explanation.
    """
    model = genai.GenerativeModel("gemini-2.5-pro")
    response = model.generate_content(prompt)
    return response.text

def evaluate_answer(question, answer):
    """Evaluate candidate answer using Gemini"""

    # Ensure inputs are not empty
    if not question.strip() or not answer.strip():
        return "⚠️ Question or Answer is empty. Please provide valid input."

    # Prompt to send to Gemini
    eval_prompt = f"""
    Evaluate this interview response:
    Question: {question}
    Answer: {answer}

    Provide:
    1. Score out of 10
    2. Strengths
    3. Weaknesses
    4. Improvement Tips
    """

    # Initialize model
    model = genai.GenerativeModel("gemini-2.5-pro")
    
    # Generate response
    response = model.generate_content(eval_prompt)

    # Safely access model output
    if response.candidates and response.candidates[0].content.parts:
        return response.candidates[0].content.parts[0].text
    else:
        return "⚠️ No feedback was generated by the model. Try rephrasing your answer."
    
    # ==============================
# STEP 3: Input Modes
# ==============================
def get_text_input():
    return input("Type your answer: ")

def get_voice_input():
    """Capture voice and convert to text"""
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("🎤 Speak now...")
        audio = r.listen(source)
    try:
        return r.recognize_google(audio)
    except:
        return "Could not understand audio."

def get_image_input():
    """Extract text from image using OCR"""
    image_path = input("Enter image file path: ")
    img = Image.open(image_path)
    return pytesseract.image_to_string(img)

# ==============================
# STEP 4: PDF Report
# ==============================


from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from textwrap import wrap

def create_report(filename, feedback):
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter
    
    # Margins
    left_margin = 50
    top_margin = height - 50
    
    # Title
    c.setFont("Helvetica-Bold", 16)
    c.drawString(left_margin, top_margin, "AI Interview Feedback Report")
    
    # Prepare text object
    text = c.beginText()
    text.setTextOrigin(left_margin, top_margin - 40)
    text.setFont("Helvetica", 12)
    
    # Wrap long lines
    max_width = 80  # number of characters per line
    wrapped_lines = []
    for line in feedback.split("\n"):
        wrapped_lines.extend(wrap(line, width=max_width))
        wrapped_lines.append("")  # blank line between sections

    # Add lines to PDF
    for line in wrapped_lines:
        text.textLine(line)

    c.drawText(text)
    c.save()

# ==============================
# STEP 5: Main Logic
# ==============================
def main():
    print("\n=== AI Multi-Modal Interview Bot (Gemini) ===")
    role = input("Enter Job Role: ")

    # Generate questions
    print("\nGenerating interview questions...")
    questions = generate_questions(role)
    print("\n--- Interview Questions ---")
    print(questions)

    # Choose input mode
    print("\nSelect Input Mode:")
    print("1. Text")
    print("2. Voice")
    print("3. Image")
    choice = input("Enter choice (1/2/3): ")

    if choice == "1":
        answer = get_text_input()
    elif choice == "2":
        answer = get_voice_input()
    elif choice == "3":
        answer = get_image_input()
    else:
        print("Invalid choice!")
        return

    # Evaluate answer
    print("\nEvaluating your answer...")
    feedback = evaluate_answer("Sample Question", answer)
    print("\n--- AI Feedback ---")
    print(feedback)

    # Save PDF report
    create_report("Interview_Report.pdf", feedback)
    print("\n✅ Interview completed! Check 'Interview_Report.pdf' for details.")

if __name__ == "__main__":
    main()


=== AI Multi-Modal Interview Bot (Gemini) ===

Generating interview questions...

--- Interview Questions ---
Describe the most complex technical project you've worked on. What were the biggest challenges, and how did you overcome them?

Select Input Mode:
1. Text
2. Voice
3. Image

Evaluating your answer...

--- AI Feedback ---
This is an excellent interview response. It's well-structured, detailed, and effectively showcases a wide range of technical and problem-solving skills.

### 1. Score: 9/10

This is a top-tier answer that would impress in almost any technical interview. It's just shy of a perfect 10 because it lacks business context and quantifiable metrics, which would elevate it from a great technical story to a great engineering impact story.

---

### 2. Strengths

*   **Excellent Structure:** The answer follows a clear and effective "Problem -> Action -> Result" (a variation of the STAR method) format. It explicitly calls out the "Biggest Challenges" and "How I Overcame T